In [ ]:
pip install 

In [35]:
import os
import scipy.io
import re
import numpy as np
from scipy.stats import kurtosis, skew
import matplotlib.pyplot as plt
import pywt
import gc

In [26]:
# Main directory
main_dir =r'D:\Cutting_Tools_Code\Cutting_Tools_Data\Vibration_500mV'

In [27]:
# List of specific keys
keys = ['BF', 'BFI', 'GF', 'GFI', 'N', 'NI', 'TF']

# Initialize the mat_data dictionary with these keys
mat_data = {key: [] for key in keys}

In [28]:
i = 1
for sub_dir in os.listdir(main_dir):
    sub_dir_path = os.path.join(main_dir, sub_dir)
    if os.path.isdir(sub_dir_path):
        mat_data[sub_dir] = []  # Initialize if not already done

        for file in os.listdir(sub_dir_path):
            if file.endswith('.mat'):
                
                file_path = os.path.join(sub_dir_path, file)
                mat_contents = scipy.io.loadmat(file_path)
                signal = None
                if 'signal' in mat_contents:
                    signal = mat_contents['signal'].squeeze()  # Ensuring signal is 1D
                elif 'signals' in mat_contents:
                    signal = mat_contents['signals'].squeeze()  # Ensuring signals is 1D


                mat_data[sub_dir].append(signal[0])
                # # Divide the signal into two parts
                # mid_index = len(signal) // 2
                # part1 = signal[:mid_index]
                # part2 = signal[mid_index:]

                # # Append the two parts to the mat_data dictionary
                # mat_data[sub_dir].append(part1)
                # mat_data[sub_dir].append(part2)

                i += 1


In [29]:
for key in mat_data.keys():
    # This concatenates all arrays in the list into one array, then flattens it
    mat_data[key] = np.concatenate(mat_data[key]).flatten()

In [30]:
print(len(mat_data['BF']))
print(len(mat_data['BFI']))
print(len(mat_data['GF']))
print(len(mat_data['GFI']))
print(len(mat_data['N']))
print(len(mat_data['NI']))
print(len(mat_data['TF']))


1024000
1024000
1024000
1024000
1024000
768000
1024000


In [31]:
segment_length = 25600  # Length of each segment
overlap = int(segment_length * 0.34)  # 75% overlap

segmented_data = {}  # Dictionary to store segmented signals

for key, signal in mat_data.items():
    # Initialize a list to store segments for this key
    segments = []
    
    # Start index for the first segment
    start_idx = 0
    
    # Calculate the number of segments we can extract
    while start_idx + segment_length <= len(signal):
        # Extract a segment
        segment = signal[start_idx:start_idx + segment_length]
        segments.append(segment)
        
        # Update start index for the next segment, accounting for the new overlap
        # This effectively moves the start index by 25% of the segment length each time
        start_idx += segment_length - overlap  # Move start index by 25% of segment length for 75% overlap
    
    # Store the list of segments in the new dictionary
    segmented_data[key] = segments